# 3.9.3: Baseball 

---

<br>

*Modeling and Simulation in Python*

Copyright 2021 Allen Downey, (License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/))

Revised, Mike Augspurger (2021-present)

<br>

---

In [1]:
#@title
# Import necessary libraries
from os.path import basename, exists
from os import mkdir

def download(url,folder):
    filename = folder + basename(url)
    if not exists(folder):
        mkdir(folder)
    # fetches the file at the given url if it is not already present
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Notebooks/'
        + 'ModSimPy_Functions/modsim.py', 'ModSimPy_Functions/')
download('https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Notebooks/'
        + 'ModSimPy_Functions/chap08.py', 'ModSimPy_Functions/')

from ModSimPy_Functions.modsim import *
from ModSimPy_Functions.chap08 import *
import pandas as pd
import numpy as np

Downloaded ModSimPy_Functions/modsim.py
Downloaded ModSimPy_Functions/chap08.py


## The Manny Ramirez Problem

Manny Ramirez is a former member of the Boston Red Sox (an American
baseball team) who was notorious for his relaxed attitude and taste for practical jokes. Our objective in this chapter is to solve the following Manny-inspired problem:  What is the minimum effort required to hit a home run in Fenway Park?

<br>

Fenway Park is a baseball stadium in Boston, Massachusetts. One of its
most famous features is the "Green Monster", which is a wall in left
field that is unusually close to home plate, only 310 feet away. To
compensate for the short distance, the wall is unusually high, at 37
feet.

### Creating a system

Let's add these two parameters to our `params` object (`wall_distance` and `wall_height`, in meters):

In [2]:
# Convert feet to meters
feet_to_meter = 0.3048
params = dict(
    x = 0,          # m
    y = 1,          # m
    angle = 45,     # degree
    speed = 40,  # m / s

    mass = 145e-3,    # kg 
    diameter = 73e-3, # m 
    C_d = 0.33,       # dimensionless

    rho = 1.2,      # kg/m**3
    g = 9.8,        # m/s**2
    t_end = 10,    # s
    
    wall_distance= 310*feet_to_meter,     # m
    wall_height= 37 * feet_to_meter)      # m

We'll need to adjust the `make_system` function to include our new parameters:

In [3]:
def make_system(params):
    
    x, y, angle, speed, mass, diameter, C_d, rho, g, t_end, wall_distance, wall_height = params.values()
    
    # compute x and y components of velocity
    vx, vy = angle_to_components(speed,angle)
    
    # make the initial state
    init = pd.Series(dict(x=x, y=y, vx=vx, vy=vy))
    
    # compute the frontal area
    area = np.pi * (diameter/2)**2

    return dict(C_d=C_d, rho=rho, g=g, t_end=t_end,
                  mass=mass,wall_distance=wall_distance,
                  wall_height=wall_height,init = init,
                  area = area)

The answer we want is the minimum speed at which a ball can leave home plate and still go over the Green Monster. We'll proceed in the
following steps:

<br>

1.  For a given speed, we'll find the optimal *launch angle*, that is, the angle the ball should leave home plate to maximize its height when it reaches the wall.

2.  Then we'll find the minimum speed that clears the wall, given that it has the optimal launch angle.



### Part 1

As a first step, write an `event_func` that stops the simulation when the ball reaches the wall at `wall_distance`, which is a parameter in `params`.
Test your function with the initial conditions (it should return `wall_distance`):

In [6]:
# Define the event_func that will make the simulation stop 
# when the ball reaches the outfield wall

def event_func(t,state,system):


In [10]:
# Test event function
system = make_system(params)
event_func(0,system['init'],system)

94.488

In [9]:
# Run a simulation with event_func and make sure the final value of x is wall_distance
results, details = run_solve_ivp(system, slope_func,
                                     events=event_func)
results.iloc[-1].x

94.488

### Part 2

Now we need a function that takes a launch angle and returns the height of the baseball when it reaches the wall.  'height_func` should:
* take a launch angle and `params`
* update params with the new launch angle (use `copy` and `update`)
* create a new system
* run a simulation with the new system
* stop the simulation at the wall
* return the height of the baseball when it reaches the wall.

<br>

Test your function with a 45 degree angle and normal parameters: you should find that the height function returns a value of -6.9 m (Did you remember to make the return value negative so we could "minimize" it? 😀)

In [ ]:
# Define height function
# If you are unsure how to do this, follow the steps listed above
# one by one!
def height_func(angle,params):


In [ ]:
# Test height_function
height_func(45,params)

### Part 3

We now have a function that will tell us the height of the ball when it reaches the wall.  If we use this function in `minimize_scalar`, we can find the angle that will maximize this height.  Is it higher or lower than the angle that maximizes range?

In [ ]:
# Find the optimal angle
# Set the initial velocity to 40
params['speed'] = 40
max_ang = spo.minimize_scalar(height_func,args=params)
print(max_ang.x, max_ang.fun)

The angle that maximizes the height at the wall is a little higher than the angle that maximizes range.  



<br> We have the tool to optimize the angle for a given speed.  Now we will use `minimize_scalar` again and a function that adjusts the initial speed to find the lowest speed whose maximum height clears the wall.

<br>

Write an `velocity_function` that takes a speed and a `system` object as parameters.  It should use `minimize_scalar` to find the angle that will produce the highest possible height of the ball at the wall, for the given speed.  Then it should return the difference between that optimal height and `wall_height`, which is a parameter in `params`.


In [ ]:
# Define the velocity function

def velocity_func(speed,params):
    # update params using 'update' and 'copy'

    
    # Use `minimize_scalar` to find the angle that gives the maximum 
    # height at the wall for this 'speed' (return to 'max_ang')


    # print out the Speed, angle, and maximum height for this speed
    print("Speed = ",speed,"Angle = ", max_ang.x, "Height = ", -max_ang.fun)
    
    # Return the difference between the top of the wall and the optimal height
    # of the ball at the wall: this is what we want to minimize
    # Remember that max_ang.fun is defined as a negative


In [ ]:
# Test function: should get a result of about 4 m
velocity_func(40,params)

Then use `root_scalar` to find the answer to the problem, the minimum speed that gets the ball out of the park.

In [ ]:
# Now call minimize_scalar to find the angle and minimum speed to clear wall
# We're "bounding" our possible speeds between 10 and 70 m/s to avoid
# some computational problems
best = spo.minimize_scalar(vel_func,args=params, method='bounded',bounds=[10,70])
print(best.x, best.fun)


Part 4

✅ A. Look at the print out for speed, angle, and maximum height.  What is happening with each iteration of `velocity_func`?

✅ ✅ Answer A here.

✅ B. Both `root_scalar` and `minimize_scalar` search for a return value (either 0 or a minimum) by "iterating": that is, running the simulation, checking the result, and guessing at the next parameter.   Is there a way we could use `root_scalar` (instead of `minimize_scalar`) to find the speed value that would just get over the wall?  What would our "velocity_func" `return` instead of abs(wall_height + final_y)?

✅ ✅ Answer B here.